In [1]:
from os import getcwd
from pathlib import Path

import flopy as fp
import numpy as np

In [2]:
workspace = Path(getcwd(), "model_p1a")
workspace

WindowsPath('c:/Users/buckl/Documents/Github/flopy_mf6_work/notebooks/implementing/gwf_bcf2ss/model_p1a')

In [3]:
sim_name = "gwf_bcf2ss"
sim = fp.mf6.MFSimulation(
    sim_name=sim_name,
    version="mf6",
    exe_name=Path("C:\\Users\\buckl\\Documents\\Software\\mf6.4.1\\bin\\mf6.exe"),
    sim_ws=workspace
)
sim

sim_name = gwf_bcf2ss
sim_path = c:\Users\buckl\Documents\Github\flopy_mf6_work\notebooks\implementing\gwf_bcf2ss\model_p1a
exe_name = C:\Users\buckl\Documents\Software\mf6.4.1\bin\mf6.exe

###################
Package mfsim.nam
###################

package_name = mfsim.nam
filename = mfsim.nam
package_type = nam
model_or_simulation_package = simulation
simulation_name = gwf_bcf2ss



In [4]:
fp.mf6.ModflowTdis(
    simulation=sim,
    time_units="days",
    nper=2,
    perioddata=[
        (1.0, 1, 1.0),
        (1.0, 1, 1.0)
    ],
    filename=f"{sim_name}.tdis",
    pname="tdis"
)

package_name = tdis
filename = gwf_bcf2ss.tdis
package_type = tdis
model_or_simulation_package = simulation
simulation_name = gwf_bcf2ss

Block options
--------------------
time_units
{internal}
('days')


Block dimensions
--------------------
nper
{internal}
(2)


Block perioddata
--------------------
perioddata
{internal}
(rec.array([(1., 1, 1.), (1., 1, 1.)],
          dtype=[('perlen', '<f8'), ('nstp', '<i4'), ('tsmult', '<f8')]))



In [5]:
fp.mf6.ModflowIms(
    simulation=sim,
    outer_dvclose=1e-6,
    outer_maximum=500,
    inner_maximum=100,
    inner_dvclose=1e-6,
    rcloserecord=[0.001, "strict"],
    linear_acceleration="cg",
    relaxation_factor=0.97,
    filename=f"{sim_name}.ims",
    pname="ims"
)

package_name = ims
filename = gwf_bcf2ss.ims
package_type = ims
model_or_simulation_package = simulation
simulation_name = gwf_bcf2ss

Block nonlinear
--------------------
outer_dvclose
{internal}
(1e-06)

outer_maximum
{internal}
(500)


Block linear
--------------------
inner_maximum
{internal}
(100)

inner_dvclose
{internal}
(1e-06)

rcloserecord
{internal}
(rec.array([('inner_rclose', 0.001, 'strict')],
          dtype=[('inner_rclose_label', 'O'), ('inner_rclose', '<f8'), ('rclose_option', 'O')]))

linear_acceleration
{internal}
('cg')

relaxation_factor
{internal}
(0.97)



In [6]:
model = fp.mf6.ModflowGwf(
    simulation=sim,
    modelname=sim_name,
    model_nam_file=f"{sim_name}.nam",
    # from original nam file
    save_flows=True
)
model

name = gwf_bcf2ss
model_type = gwf6
version = mf6
model_relative_path = .


In [7]:
fp.mf6.ModflowGwfdis(
    model=model,
    length_units="feet",
    nlay=2,
    nrow=10,
    ncol=15,
    delr=500.0,
    delc=500.0,
    top=150.0,
    botm = [
        50.0,
        -50.0
    ],
    filename=f"{sim_name}.dis",
    pname="dis"
)

package_name = dis
filename = gwf_bcf2ss.dis
package_type = dis
model_or_simulation_package = model
model_name = gwf_bcf2ss

Block options
--------------------
length_units
{internal}
('feet')


Block dimensions
--------------------
nlay
{internal}
(2)

nrow
{internal}
(10)

ncol
{internal}
(15)


Block griddata
--------------------
delr
{constant 500.0}

delc
{constant 500.0}

top
{constant 150.0}

botm
Layer_1{constant 50.0}
Layer_2{constant -50.0}



In [8]:
l1_wetdry = np.ones((10, 15))
l1_wetdry[(2, 7), 3] = -1
l1_wetdry[:, 8:] = -1
l1_wetdry *= 2

fp.mf6.ModflowGwfnpf(
    model=model,
    rewet_record=[
        'wetfct', 1.0,
        'iwetit', 1,
        'ihdwet', 0
    ],
    save_specific_discharge=True,
    icelltype=[1, 0],
    k=[10.0, 5.0],
    k33=0.1,
    wetdry=[
        l1_wetdry,
        0.0
    ],
    filename=f"{sim_name}.npf",
    pname="npf"
)

package_name = npf
filename = gwf_bcf2ss.npf
package_type = npf
model_or_simulation_package = model
model_name = gwf_bcf2ss

Block options
--------------------
rewet_record
{internal}
(rec.array([('wetfct', 1., 'iwetit', 1, 'ihdwet', 0)],
          dtype=[('wetfct_label', 'O'), ('wetfct', '<f8'), ('iwetit_label', 'O'), ('iwetit', '<i4'), ('ihdwet_label', 'O'), ('ihdwet', '<i4')]))

save_specific_discharge
{internal}
(True)


Block griddata
--------------------
icelltype
Layer_1{constant 1}
Layer_2{constant 0}

k
Layer_1{constant 10.0}
Layer_2{constant 5.0}

k33
{constant 0.1}

wetdry
Layer_1{internal}
(array([[ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2., -2., -2., -2., -2., -2.,
        -2., -2.],
       [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2., -2., -2., -2., -2., -2.,
        -2., -2.],
       [ 2.,  2.,  2., -2.,  2.,  2.,  2.,  2., -2., -2., -2., -2., -2.,
        -2., -2.],
       [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2., -2., -2., -2., -2., -2.,
        -2., -2.],
       [ 2.,  2.,  2.,

In [9]:
fp.mf6.ModflowGwfic(
    model=model,
    strt=0.0,
    filename=f"{sim_name}.ic",
    pname="ic"
)

package_name = ic
filename = gwf_bcf2ss.ic
package_type = ic
model_or_simulation_package = model
model_name = gwf_bcf2ss

Block griddata
--------------------
strt
{constant 0.0}



In [10]:
def make_riv_iter():
    for i in range(10):
        yield ((1, i, 14), 0.0, 10000.0, -5.0)

fp.mf6.ModflowGwfriv(
    model=model,
    maxbound=10,
    stress_period_data=list(make_riv_iter()),
    filename=f"{sim_name}.riv",
    pname="riv_0"
)

package_name = riv_0
filename = gwf_bcf2ss.riv
package_type = riv
model_or_simulation_package = model
model_name = gwf_bcf2ss

Block dimensions
--------------------
maxbound
{internal}
(10)


Block period
--------------------
stress_period_data
{0: <flopy.mf6.data.mfdataplist.PandasListStorage object at 0x000002832D4EC050>}


In [11]:
model.package_dict['riv_0'].stress_period_data.get_data(key=0)

rec.array([((1, 0, 14), 0., 10000., -5.), ((1, 1, 14), 0., 10000., -5.),
           ((1, 2, 14), 0., 10000., -5.), ((1, 3, 14), 0., 10000., -5.),
           ((1, 4, 14), 0., 10000., -5.), ((1, 5, 14), 0., 10000., -5.),
           ((1, 6, 14), 0., 10000., -5.), ((1, 7, 14), 0., 10000., -5.),
           ((1, 8, 14), 0., 10000., -5.), ((1, 9, 14), 0., 10000., -5.)],
          dtype=[('cellid', 'O'), ('stage', '<f8'), ('cond', '<f8'), ('rbot', '<f8')])

In [12]:
fp.mf6.ModflowGwfwel(
    model=model,
    maxbound=2,
    stress_period_data={
        1: [
            ((1, 2, 3), -35000.0),
            ((1, 7, 3), -35000.0)
        ]
    },
    filename=f"{sim_name}.wel",
    pname="wel_0"
)

package_name = wel_0
filename = gwf_bcf2ss.wel
package_type = wel
model_or_simulation_package = model
model_name = gwf_bcf2ss

Block dimensions
--------------------
maxbound
{internal}
(2)


Block period
--------------------
stress_period_data
{1: <flopy.mf6.data.mfdataplist.PandasListStorage object at 0x000002832DA3BA10>}


In [13]:
model.package_dict['wel_0'].stress_period_data.get_data(key=1)

rec.array([((1, 2, 3), -35000.), ((1, 7, 3), -35000.)],
          dtype=[('cellid', 'O'), ('q', '<f8')])

In [14]:
fp.mf6.ModflowGwfrcha(
    model=model,
    recharge=0.004,
    filename=f"{sim_name}.rcha",
    pname="rcha_0"
)

package_name = rcha_0
filename = gwf_bcf2ss.rcha
package_type = rcha
model_or_simulation_package = model
model_name = gwf_bcf2ss

Block options
--------------------
readasarrays
{internal}
(True)


Block period
--------------------
irch
None
recharge
{constant 0.004}

aux
None


In [15]:
fp.mf6.ModflowGwfoc(
    model=model,
    budget_filerecord=[f"{sim_name}.cbc"],
    head_filerecord=[f"{sim_name}.hds"],
    saverecord=[
        ("head", "all"),
        ("budget", "all")
    ],
    filename=f"{sim_name}",
    pname="oc"
)

package_name = oc
filename = gwf_bcf2ss
package_type = oc
model_or_simulation_package = model
model_name = gwf_bcf2ss

Block options
--------------------
budget_filerecord
{internal}
(rec.array([('gwf_bcf2ss.cbc',)],
          dtype=[('budgetfile', 'O')]))

head_filerecord
{internal}
(rec.array([('gwf_bcf2ss.hds',)],
          dtype=[('headfile', 'O')]))


Block period
--------------------
saverecord
{internal}
(rec.array([('head', 'all', None), ('budget', 'all', None)],
          dtype=[('rtype', 'O'), ('ocsetting', 'O'), ('ocsetting_data', 'O')]))

printrecord
None


In [16]:
sim.check()

Checking model "gwf_bcf2ss"...

gwf_bcf2ss MODEL DATA VALIDATION SUMMARY:
  No errors or warnings encountered.

  Checks that passed:
    npf package: zero or negative horizontal hydraulic conductivity values
    npf package: vertical hydraulic conductivity values below checker threshold of 1e-11
    npf package: vertical hydraulic conductivity values above checker threshold of 100000.0
    npf package: horizontal hydraulic conductivity values below checker threshold of 1e-11
    npf package: horizontal hydraulic conductivity values above checker threshold of 100000.0
    riv_0 package: BC indices valid
    riv_0 package: not a number (Nan) entries
    riv_0 package: BC in inactive cells
    wel_0 package: BC indices valid
    wel_0 package: not a number (Nan) entries
    wel_0 package: BC in inactive cells

Checking for missing simulation packages...


In [17]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims...
  writing model gwf_bcf2ss...
    writing model name file...
    writing package dis...
    writing package npf...
    writing package ic...
    writing package riv_0...
    writing package wel_0...
    writing package rcha_0...
    writing package oc...


In [19]:
success, buff = sim.run_simulation()
assert success, "MODFLOW did not terminate normally!"

FloPy is using the following executable to run the model: ..\..\..\..\..\..\Software\mf6.4.1\bin\mf6.exe
                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                        VERSION 6.4.1 Release 12/09/2022

   MODFLOW 6 compiled Dec 09 2022 18:18:37 with Intel(R) Fortran Intel(R) 64
   Compiler Classic for applications running on Intel(R) 64, Version 2021.7.0
                             Build 20220726_000000

This software has been approved for release by the U.S. Geological 
Survey (USGS). Although the software has been subjected to rigorous 
review, the USGS reserves the right to update the software as needed 
pursuant to further analysis and review. No warranty, expressed or 
implied, is made by the USGS or the U.S. Government as to the 
functionality of the software and related material nor shall the 
fact of release constitute any such warranty. Furthermore, the 
software is released on condition that neither the US

In [20]:
sim.delete_output_files()